In [ ]:
# HURST EXPONENT TEST
"""
The goal of the Hurst Exponent is to provide us with a scalar value that will help us to identify
(within the limits of statistical estimation) whether a series is mean reverting, random walking or trending.

This is done by using the variance of a log price series to assess the rate of diffusive behaviour.

CONDITIONS FOR COMMODITY BEHAVIOUR
• H < 0:5 - The time series is mean reverting
• H = 0:5 - The time series is a Geometric Brownian Motion
• H > 0:5 - The time series is trending

The Hurst Exponent also describes the extent to which a series behaves in the manner categorised. 
For instance, a value of H near 0 is a highly mean reverting series, 
while for H near 1 the series is strongly trending.

NOTE: if H > 0.55:
    print("Strong trending behavior")
elif H > 0.5:
    print("Weak trending behavior") 
elif H == 0.5:
    print("Perfect geometric Brownian motion")
elif H > 0.45:
    print("Weak mean reversion (your case)")
else:
    print("Strong mean reversion")
"""

In [ ]:
#Import Necessary Modules
from __future__ import print_function

from scipy import stats

from datetime import datetime, timedelta

import MetaTrader5 as mt5
from config import *
import pandas as pd
import plotly.express as px
import numpy as np

In [62]:
#Check connection to MT5
result = mt5.initialize()
if login_number == mt5.account_info().login and result is True:
    print("Connection to MetaTrader5 established")
else:
    print("Failed to connect at account #{}, error code: {}".format(login_number, mt5.last_error()))
    mt5.shutdown()

Connection to MetaTrader5 established


In [69]:
#Get Amazon OHLC Data from today back 730days
#OHLC DATA
rates = mt5.copy_rates_range("AMZN.NAS", mt5.TIMEFRAME_D1, datetime.now() - timedelta(days=730), datetime.now())

ohlc_data = pd.DataFrame(rates)

fig = px.line(ohlc_data, x=ohlc_data['time'], y=ohlc_data['close'], title='AMAZN.NAS Closing Prices - Last 730 Days')
fig.show()

data = ohlc_data['close'].values
ohlc_data
print(data)

[187.85 183.69 183.13 182.45 186.17 180.78 179.73 182.48 183.15 181.62
 187.03 183.36 167.64 160.5  161.95 162.6  165.88 166.89 166.74 170.26
 170.01 177.53 176.99 177.96 178.8  180.04 176.17 176.95 175.48 172.99
 170.69 172.1  178.51 175.95 173.2  177.67 171.34 175.38 179.61 184.66
 186.93 186.39 184.71 186.66 186.22 189.85 191.58 194.01 193.99 192.46
 191.21 187.78 186.09 185.09 184.69 182.02 186.46 180.83 182.55 185.14
 186.54 188.71 187.52 187.65 186.88 187.56 188.86 189.05 189.65 184.73
 186.32 187.83 188.33 190.81 192.74 186.03 197.83 195.75 199.39 207.15
 210.07 208.12 206.79 208.8  214.07 211.59 202.47 201.63 204.54 202.81
 198.27 196.99 201.35 207.8  205.66 207.83 210.53 213.33 218.01 220.46
 226.92 225.94 224.93 230.31 229.   227.35 232.87 231.03 220.55 223.29
 225.06 224.82 223.63 221.2  220.13 224.15 227.45 222.03 222.2  218.78
 218.29 217.59 223.34 220.48 225.83 230.55 234.98 235.3  234.76 235.48
 238.18 236.92 234.44 237.68 237.43 242.12 236.17 238.63 229.22 233.11
 232.7

In [ ]:
def hurst(ts):
    """
    Returns the Hurst exponent of a time series 'ts'
    """
    # Ensure we have a numpy array and remove any NaN/inf values
    ts = np.array(ts, dtype=float)
    ts = ts[np.isfinite(ts)]
    
    if len(ts) < 100:
        raise ValueError("Time series too short")
    
    lags = range(2, min(100, len(ts)//2))  # Adjust lags based on data length
    tau = []
    valid_lags = []
    
    for lag in lags:
        if lag >= len(ts):
            continue
            
        # Calculate differences
        differences = ts[lag:] - ts[:-lag]
        std_diff = np.std(differences, ddof=1)
        
        # Only include if standard deviation is positive
        if std_diff > 1e-10:  # Small threshold to avoid numerical issues
            tau.append(np.sqrt(std_diff))
            valid_lags.append(lag)
    
    if len(tau) < 2:
        return np.nan
    
    # Use valid data for regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(
        np.log(valid_lags), 
        np.log(tau)
    )
    
    return slope * 2.0

print("Hurst(AMZN):     %s " % hurst(data))

"""AMAZN had 0.4657451364791901 (calculated, txtbook had 0.454337476553) Interestingly, Amazon has H also close to 0.5 indicating that it is similar to a GBM,
at least for the sample period we're making use of!"""

Hurst(AMZN):     0.4657451364791901 
